## 1. Select mode
mode='train' for the training, mode='predict' for the predictions
uncomment the one you want to enable and comment the one you want to enable 



## 1. Select mode
mode='train' for the training, mode='predict' for the predictions
uncomment the one you want to enable and comment the one you want to enable 

In [ ]:
mode = 'train'
# mode = 'predict'

## 2. Import libraries

In [ ]:
# clone github repo of detr
!git clone https://github.com/facebookresearch/detr.git   

# general libraries
import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.autonotebook import tqdm
import re
import pydicom
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot as plt
from PIL import Image

# torch.
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader, Dataset

# sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection

# CV
import cv2

# DETR FUCNTIONS FOR LOSS
import sys
sys.path.append('./detr/')

from detr.models.matcher import HungarianMatcher
from detr.models.detr import SetCriterion

# albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

# Glob
from glob import glob

# ensembling 
!pip install ensemble_boxes
from tqdm import tqdm
from ensemble_boxes import *

# mAP
!pip install map_boxes 
from map_boxes import mean_average_precision_for_boxes

In [ ]:
# thoracic abnormalities
CLASSES = [
    'Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation',
    'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 
    'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis', 'No Finding'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]